# Analyse point source density 

In [ ]:
%matplotlib inline

import re
import os
import sys
import glob
import pickle
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import oqman.man.model as model

from oqman.man.single.areas import get_rates_density
from oqman.man.single.points import get_rates_density
from openquake.hazardlib.const import TRT

## List of models to be analysed

In [ ]:
path = '/Users/mpagani/NC/Hazard_Charles/Documents/Presentations/2017/0417_SSA/notebooks/pkl/*.pkl'
modell = set(['als07', 'aus12', 'ca10h', 'ca10r', 'cub03', 'ear16', 'em16a',
              'em16f', 'emc15', 'lea02', 'nzl10', 'res12', 'sar16', 'sea07',
              'soa10', 'twn15', 'usa08'])
#modell = set(['cub03'])
#modell = set(['nzl10'])
#modell = set(['usa08'])
#modell = set(['als07'])
modell = set(['nzl10'])

In [ ]:
modd = {}
for fname in glob.glob(path):
    slist = re.split('\.',  os.path.basename(fname))
    if re.search('[a-z]*[0-9]*\.pkl', fname) and slist[0] in modell:
        print ('>>', fname, end=' ')
        # Creating output filename 
        dname = os.path.dirname(fname)
        slist = re.split('_', re.split('\.',  os.path.basename(fname))[0])
        outf = os.path.join(dname, '{0:s}_cellsize_sources.pkl'.format(slist[0]))
        if os.path.exists(outf):
            print ('loading ', outf)
            fou = open(outf, 'rb')
            csize = pickle.load(fou)
            print (len(csize))
            fou.close()
            modd[slist[0]] = {'csize': csize}
        else:
            print ('skipping since file exists')

## Read data for the different models

In [ ]:
%%time
for fname in glob.glob(path):
    slist = re.split('\.',  os.path.basename(fname))
    if re.search('[a-z]*[0-9]*\.pkl', fname) and slist[0] in modell:
        print ('>>', fname, end=' ')
        mod, point, info, sidx = model.load(fname)      
        print (len(point), end=' ')
        print ('done')
        if slist[0] not in modd:
            modd[slist[0]] = {}
        modd[slist[0]]['model'] = mod
        modd[slist[0]]['info'] = info
        modd[slist[0]]['point'] = point
        modd[slist[0]]['sidx'] = sidx

## Plotting

In [ ]:
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(1, 1, 1)

mpl.rcParams['xtick.labelsize'] = 14
mpl.rcParams['ytick.labelsize'] = 14

bins = np.logspace(-10, -2, 17)

cnt = 1
lbls = []
xs = []
his = {}

for key in sorted(modd.keys()):
    # Check if the model contains point sources
    if 'PointSource' in modd[key]['info']['srcs_mmax']:
        print ('>>>>>>>>', key)
        
        fltr = 'Active Shallow Crust'; sxx = 0.35
        print (len(modd[key]['csize']))
        areas = modd[key]['csize']

        vals = []
        for kkk in areas:
            vals.append(areas[kkk])

        print ('Cell sizes:', min(vals), max(vals))

        point = modd[key]['point'] 
        sidx = modd[key]['sidx']

        print ('Length of point dict', len(point))

        dend, _ = get_rates_density(point, sidx, mmint=5.5, trt=fltr, area=areas)

        dens = []
        for _, value in dend.items():
            dens.append(value)

        print ('-- len of density points', len(dens))

        his, _ = np.histogram(dens, bins)
        print ('Density:', min(dens), max(dens))

        imin = np.min(np.nonzero(his>0))
        imax = np.max(np.nonzero(his>0))

        lbls.append(key)
        xs.append(cnt)
        plt.plot([cnt, cnt], [min(dens), max(dens)],
                  linewidth=5, color='grey', alpha=.5)
        
        # plotting histogram
        dlt = 1
        nrm = his[imin:imax+dlt]/float(sum(his[imin:imax+dlt]))

        plt.barh(bins[imin:imax+dlt], 
                 nrm, 
                 height=np.diff(bins[imin:imax+2])*0.8, 
                 left=cnt+0.1, 
                 edgecolor='black',
                 align='edge')

        for y, x, h in zip(bins[imin:imax+dlt], nrm, his[imin:imax+dlt]):
            plt.text(cnt-sxx, y, '{0:>5d}'.format(h))

        # updating counter
        cnt += 1

plt.yscale('log')
plt.ylabel('Eqks rate [1/[km2*yr]]', fontsize=14)
plt.grid(linestyle='--')
plt.ylim([1e-10, 1e-2])
aa = plt.xticks(xs, lbls, rotation='vertical')

##### 